This notebook is used to train varous PyTorch models.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
""" Import your model here """
from models.dropoutModel4 import DropoutModel as Model
""" Iteration """
current_it = 2

In [3]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
from torchvision.transforms import transforms

from image_datasets.imagedataset import ImageDataset

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


### Hyperparameters

In [5]:
num_epochs = 5
batch_size = 10

learning_rate = 0.001
momentum = 0.9

### Data

In [6]:
norm_transform = transforms.Normalize(
    (132.3501, 127.2977, 131.0638),
    (55.5031, 62.3274, 64.1869)
)

In [7]:
dataset = ImageDataset(transform=norm_transform)
train_size = int(0.75*len(dataset))
test_size  = len(dataset) - train_size
train, test = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test, batch_size=batch_size, shuffle=False)

### Loss & optimizer

In [8]:
model = Model().to(device)
criterion = nn.CrossEntropyLoss() # Includes softmax
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=1e-5)


In [9]:
if current_it != 1:
    model.load_state_dict(torch.load("./model_states/"+model.name+"_"+str(current_it-1)+".pth"))

FileNotFoundError: [Errno 2] No such file or directory: './model_states/model_v4_1.pth'

### Training loop

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward
        outputs = model(images)
        #print(outputs)
        loss = criterion(outputs, labels)

        # Backward:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 500 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}")

### Saving

Change the name of the saved file!

In [ ]:
torch.save(model.state_dict(), "./model_states/" + model.name+"_"+str(current_it)+".pth")

### Validation

In [ ]:
with torch.no_grad(): 

    labs = np.empty(0, dtype=object) # List for all labels and predictions in test set
    pred = np.empty(0, dtype=object)
    for (images, labels) in test_loader:
        images = images.float().to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)     # Retrieving index of maximum output
        labs = np.append(labs, np.array(labels.cpu()))
        pred = np.append(pred, np.array(predicted.cpu()))

In [ ]:
# Total accuracy
accuracy = np.round((labs == pred).mean(), 3)

# In-class accuracy:
num_classes = 29
class_accuracy = np.zeros(num_classes)
for i in range(num_classes):
    class_ind = (labs == i) # Indices of class i in labs and pred
    class_accuracy[i] = np.mean(labs[class_ind] == pred[class_ind])

print(f"Accuracy: {accuracy}")

In [ ]:
plt.style.use("seaborn");
plt.ylim([0.75, 1])
plt.bar(range(num_classes), class_accuracy, label="Class prediction accuracy")
plt.hlines(accuracy, 0, num_classes, colors="red", linestyles="--", label="Average (total) accuracy")
plt.xlabel("Class")
plt.ylabel("Accuracy")
plt.legend()